In [1]:
import pandas as pd
from sklearn.neighbors import BallTree
import h3

In [2]:
df = pd.read_csv('./Data/NYPD_Complaint_Data_Historic.csv')

C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\2002622462.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Data/NYPD_Complaint_Data_Historic.csv')


In [3]:
plutoDF = pd.read_csv('./Data/PLUTOclean.csv', index_col=0)

In [4]:
plutodict = {}
plutoDF = plutoDF.dropna(subset=['latitude'])
plutodict['QN'] = plutoDF[plutoDF['borough'] == 'QN']
plutodict['BK'] = plutoDF[plutoDF['borough'] == 'BK']
plutodict['BX'] = plutoDF[plutoDF['borough'] == 'BX']
plutodict['SI'] = plutoDF[plutoDF['borough'] == 'SI']
plutodict['MN'] = plutoDF[plutoDF['borough'] == 'MN']

In [5]:
# '''Field Name Description
# CMPLNT_NUM Randomly generated persistent ID for each complaint
# CMPLNT_FR_DT Exact date of occurrence for the reported event (or starting
#     date of occurrence, if CMPLNT_TO_DT exists)
# CMPLNT_FR_TM Exact time of occurrence for the reported event (or starting
#     time of occurrence, if CMPLNT_TO_TM exists)
# CMPLNT_TO_DT Ending date of occurrence for the reported event, if exact
#     time of occurrence is unknown
# CMPLNT_TO_TM Ending time of occurrence for the reported event, if exact
#     time of occurrence is unknown
# RPT_DT Date event was reported to police
# KY_CD Three digit offense classification code
# OFNS_DESC Description of offense corresponding with key code
# PD_CD Three digit internal classification code (more granular than
#     Key Code)
# PD_DESC Description of internal classification corresponding with PD
# code (more granular than Offense Description)
# CRM_ATPT_CPTD_CD Indicator of whether crime was successfully completed or
# attempted, but failed or was interrupted prematurely
# LAW_CAT_CD Level of offense: felony, misdemeanor, violation
# JURIS_DESC
# Jurisdiction responsible for incident. Either internal, like
# Police, Transit, and Housing; or external, like Correction,
# Port Authority, etc.
# BORO_NM The name of the borough in which the incident occurred
# ADDR_PCT_CD The precinct in which the incident occurred
# LOC_OF_OCCUR_DESC Specific location of occurrence in or around the premises;
# inside, opposite of, front of, rear of
# PREM_TYP_DESC Specific description of premises; grocery store, residence,
# street, etc.
# PARKS_NM Name of NYC park, playground or greenspace of
# occurrence, if applicable (state parks are not included)
# HADEVELOPT Name of NYCHA housing development of occurrence, if
# applicable
# X_COORD_CD X-coordinate for New York State Plane Coordinate System,
# Long Island Zone, NAD 83, units feet (FIPS 3104)
# Y_COORD_CD Y-coordinate for New York State Plane Coordinate System,
# Long Island Zone, NAD 83, units feet (FIPS 3104)
# Latitude Latitude coordinate for Global Coordinate System, WGS
# 1984, decimal degrees (EPSG 4326)
# Longitude Longitude coordinate for Global Coordinate System, WGS
# 1984, decimal degrees (EPSG 4326)'''

In [6]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,394506329,12/31/2019,17:30:00,NaN,NaN,32.0,12/31/2019,118,DANGEROUS WEAPONS,793.0,...,NaN,NaN,40.820927,-73.943324,"(40.82092679700002, -73.94332421899996)",PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,E
1,968873685,12/29/2019,16:31:00,12/29/2019,16:54:00,47.0,12/29/2019,113,FORGERY,729.0,...,NaN,NaN,40.885701,-73.861640,"(40.885701406000074, -73.86164032499995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
2,509837549,12/15/2019,18:45:00,NaN,NaN,109.0,12/29/2019,578,HARRASSMENT 2,638.0,...,M,NaN,40.742281,-73.819824,"(40.74228115600005, -73.81982408)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE HISPANIC,F
3,352454313,12/28/2019,01:00:00,NaN,NaN,47.0,12/28/2019,126,MISCELLANEOUS PENAL LAW,117.0,...,M,NaN,40.875311,-73.847545,"(40.87531145100007, -73.84754521099995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
4,248803469,09/05/2008,21:41:00,NaN,NaN,NaN,09/05/2008,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,40.698827,-73.938819,"(40.698827283, -73.938819047)",NaN,NaN,25-44,BLACK,M


In [7]:
df = df.drop(['Lat_Lon', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM', 'HADEVELOPT','X_COORD_CD','Y_COORD_CD', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM'],axis = 1, errors='ignore')
#df = df.dropna(subset=['CMPLNT_FR_DT'])

In [8]:
df = df.dropna(subset=['Latitude'])

In [9]:
plutoLookup = {}

for b in plutoDF['block'].unique():
    plutoLookup[b] = plutoDF[plutoDF['block'] == b]['bctcb2020'].max()

In [35]:
count = 0
def log(lstr = "", f=False):
    global count
    count = count + 1
    if f or count % 1000 == 0:
        file_object = open('log.txt', 'a')

        file_object.write(str(count) + " " + str(lstr) + '\n')

        file_object.close

In [11]:
dfdict = {}
dfdict['QN'] = df[df['BORO_NM'] == 'QUEENS']
dfdict['BK'] = df[df['BORO_NM'] == 'BROOKLYN']
dfdict['BX'] = df[df['BORO_NM'] == 'BRONX']
dfdict['SI'] = df[df['BORO_NM'] == 'STATEN ISLAND']
dfdict['MN'] = df[df['BORO_NM'] == 'MANHATTAN']
dfdict['null'] = df[df['BORO_NM'].isnull()]

In [12]:
def getBlock(id, borough):
    log('getBlock')
    return plutodict[borough].iloc[id]['block']
def getBct(id):
    log('getBct')
    return plutoLookup[id]

def getBlockn(id):
    log('getBlockn')
    return plutoDF.iloc[id]['block']

def getBctn(id):
    log('getBctn')
    return plutoDF.iloc[id]['bctcb2020']

def getBoro(block):
    log('getBoro')
    return plutoDF[plutoDF['block'] == block].iloc[0]['borough']

In [13]:

def h3Geo(latitude, longitude):
    log('h3')
    return h3.geo_to_h3(latitude, longitude, 10)

In [16]:
tree = BallTree(plutoDF[['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['null']['id_nearest'] = tree.query(dfdict['null'][['Latitude', 'Longitude']].values, k=1)
dfdict['null']['Block'] = dfdict['null']['id_nearest'].apply(lambda x: getBlockn(x))
dfdict['null']['bctcb2020'] = dfdict['null']['Block'].apply(lambda x: getBct(x))
dfdict['null']['BORO_NM'] = dfdict['null']['Block'].apply(lambda x: getBoro(x))

C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1474629183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _ , dfdict['null']['id_nearest'] = tree.query(dfdict['null'][['Latitude', 'Longitude']].values, k=1)
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1474629183.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdict['null']['Block'] = dfdict['null']['id_nearest'].apply(lambda x: getBlockn(x))
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1474629183.py:4: SettingWithCopyWarning: 
A 

In [17]:
count = 0
tree = BallTree(plutodict['QN'][['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['QN']['id_nearest'] = tree.query(dfdict['QN'][['Latitude', 'Longitude']].values, k=1)
count = 0
tree = BallTree(plutodict['BK'][['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['BK']['id_nearest'] = tree.query(dfdict['BK'][['Latitude', 'Longitude']].values, k=1)
count = 0
tree = BallTree(plutodict['SI'][['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['SI']['id_nearest'] = tree.query(dfdict['SI'][['Latitude', 'Longitude']].values, k=1)
count = 0
tree = BallTree(plutodict['MN'][['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['MN']['id_nearest'] = tree.query(dfdict['MN'][['Latitude', 'Longitude']].values, k=1)
count = 0
tree = BallTree(plutodict['BX'][['latitude', 'longitude']].values, leaf_size=2)
_ , dfdict['BX']['id_nearest'] = tree.query(dfdict['BX'][['Latitude', 'Longitude']].values, k=1)

C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1743111575.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _ , dfdict['QN']['id_nearest'] = tree.query(dfdict['QN'][['Latitude', 'Longitude']].values, k=1)
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1743111575.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _ , dfdict['BK']['id_nearest'] = tree.query(dfdict['BK'][['Latitude', 'Longitude']].values, k=1)
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\1743111575.py:9: SettingWithCopyWarn

In [18]:
count = 0
dfdict['QN']['Block'] = dfdict['QN']['id_nearest'].apply(lambda x: getBlock(x, 'QN'))
dfdict['QN']['bctcb2020'] = dfdict['QN']['Block'].apply(lambda x: getBct(x))
count = 0
dfdict['BK']['Block'] = dfdict['BK']['id_nearest'].apply(lambda x: getBlock(x, 'BK'))
dfdict['BK']['bctcb2020'] = dfdict['BK']['Block'].apply(lambda x: getBct(x))
count = 0
dfdict['SI']['Block'] = dfdict['SI']['id_nearest'].apply(lambda x: getBlock(x, 'SI'))
dfdict['SI']['bctcb2020'] = dfdict['SI']['Block'].apply(lambda x: getBct(x))
count = 0
dfdict['BX']['Block'] = dfdict['BX']['id_nearest'].apply(lambda x: getBlock(x, 'BX'))
dfdict['BX']['bctcb2020'] = dfdict['BX']['Block'].apply(lambda x: getBct(x))
count = 0
dfdict['MN']['Block'] = dfdict['MN']['id_nearest'].apply(lambda x: getBlock(x, 'MN'))
dfdict['MN']['bctcb2020'] = dfdict['MN']['Block'].apply(lambda x: getBct(x))

C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\353517390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdict['QN']['Block'] = dfdict['QN']['id_nearest'].apply(lambda x: getBlock(x, 'QN'))
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\353517390.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdict['QN']['bctcb2020'] = dfdict['QN']['Block'].apply(lambda x: getBct(x))
C:\Users\Brian M\AppData\Local\Temp\ipykernel_14396\353517390.py:5: SettingWithCopyWarning: 
A value is trying to be set 

In [19]:
df = pd.concat([dfdict['QN'], dfdict['BK'], dfdict['BX'], dfdict['SI'],dfdict['MN'], dfdict['null']])

In [20]:
count = 0
df['h3geo'] = df.apply(lambda x: h3Geo(x['Latitude'], x['Longitude']), axis=1)

In [21]:
# blockdict = df.groupby('Block')['bctcb2020'].max()

In [22]:
# for index, row in df[df['bctcb2020'].isnull()].iterrows():
#     df.iloc[index, df.columns.get_loc('bctcb2020')] = blockdict[row['Block']]

In [63]:
df = df.dropna(subset=['bctcb2020'])
df = df.drop(['OFNS_DESC', 'id_nearest', 'PD_CD', 'RPT_DT', 'PD_DESC', 'HOUSING_PSA', 'SUSP_AGE_GROUP','SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 
               'STATION_NAME', 'VIC_AGE_GROUP' ,'VIC_RACE','VIC_SEX'   ], axis = 1, errors='ignore')
df = df.dropna(subset=['CRM_ATPT_CPTD_CD','ADDR_PCT_CD', 'JURISDICTION_CODE', 'PATROL_BORO'])
#

In [ ]:
df = pd.get_dummies(df, prefix=['ADDR_PCT_CD_','KY_CD','LAW_CAT_CD','JURIS_DESC', ''], columns=['ADDR_PCT_CD','KY_CD', 'LAW_CAT_CD','JURIS_DESC','PATROL_BORO'])

In [47]:
df['CRIME_COMPLETED'] = df['CRM_ATPT_CPTD_CD']
df['CRIME_COMPLETED'] = df['CRIME_COMPLETED'].replace({'COMPLETED': 1, 'ATTEMPTED': 0})
df = df.drop(['CRM_ATPT_CPTD_CD'], axis=1, errors='ignore')

In [69]:
df.to_csv('./Data/NYPDcleanupfefinal2.csv', index=False)

In [68]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,BORO_NM,JURISDICTION_CODE,Latitude,Longitude,Block,bctcb2020,h3geo,...,JURIS_DESC_U.S. PARK POLICE,CRIME_COMPLETED,_PATROL BORO BKLYN NORTH,_PATROL BORO BKLYN SOUTH,_PATROL BORO BRONX,_PATROL BORO MAN NORTH,_PATROL BORO MAN SOUTH,_PATROL BORO QUEENS NORTH,_PATROL BORO QUEENS SOUTH,_PATROL BORO STATEN ISLAND
2,509837549,12/15/2019,18:45:00,QUEENS,0.0,40.742281,-73.819824,6423,5.017600e+10,8a2a100e16a7fff,...,0,1,0,0,0,0,0,1,0,0
7,134037758,12/26/2019,20:00:00,QUEENS,0.0,40.611570,-73.747365,15502,4.103201e+10,8a2a10385da7fff,...,0,1,0,0,0,0,0,0,1,0
12,946914583,12/22/2019,04:15:00,QUEENS,0.0,40.743125,-73.956068,32,5.000700e+10,8a2a100d0027fff,...,0,1,0,0,0,0,0,1,0,0
21,464928227,12/18/2019,20:30:00,QUEENS,0.0,40.698443,-73.903358,3562,5.011402e+10,8a2a100c365ffff,...,0,1,0,0,0,0,0,1,0,0
33,757750529,12/14/2019,17:00:00,QUEENS,0.0,40.750979,-73.947643,444,5.019700e+10,8a2a100d0907fff,...,0,1,0,0,0,0,0,1,0,0


In [60]:
df['VIC_SEX'].isnull().sum()

308

In [61]:
df.groupby('VIC_SEX').count()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,BORO_NM,JURISDICTION_CODE,Latitude,Longitude,PATROL_BORO,VIC_RACE,Block,...,JURIS_DESC_NYS DEPT ENVIRONMENTAL CONSERVATION,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_POLICE DEPT NYC,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_SEA GATE POLICE DEPT,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,CRIME_COMPLETED
VIC_SEX,,,,,,,,,,,,,,,,,,,,,
D,973271,973210,973267,973271,973271,973271,973271,973253,973237,973271,...,973271,973271,973271,973271,973271,973271,973271,973271,973271,973271
E,1185569,1185550,1185566,1185569,1185569,1185569,1185569,1185562,1185568,1185569,...,1185569,1185569,1185569,1185569,1185569,1185569,1185569,1185569,1185569,1185569
F,3059941,3059632,3059920,3059941,3059077,3059941,3059941,3059040,3059920,3059941,...,3059941,3059941,3059941,3059941,3059941,3059941,3059941,3059941,3059941,3059941
M,2584549,2584290,2584529,2584549,2580441,2584549,2584549,2580427,2584535,2584549,...,2584549,2584549,2584549,2584549,2584549,2584549,2584549,2584549,2584549,2584549
U,2,2,2,2,0,2,2,0,2,2,...,2,2,2,2,2,2,2,2,2,2


In [62]:
df.columns

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'BORO_NM',
       'JURISDICTION_CODE', 'Latitude', 'Longitude', 'PATROL_BORO', 'VIC_RACE',
       'VIC_SEX',
       ...
       'JURIS_DESC_NYS DEPT ENVIRONMENTAL CONSERVATION',
       'JURIS_DESC_NYS DEPT TAX AND FINANCE', 'JURIS_DESC_OTHER',
       'JURIS_DESC_POLICE DEPT NYC', 'JURIS_DESC_PORT AUTHORITY',
       'JURIS_DESC_SEA GATE POLICE DEPT', 'JURIS_DESC_STATN IS RAPID TRANS',
       'JURIS_DESC_TRI-BORO BRDG TUNNL', 'JURIS_DESC_U.S. PARK POLICE',
       'CRIME_COMPLETED'],
      dtype='object', length=193)